In [ ]:
import requests, time
import pandas as pd
import re
from bs4 import BeautifulSoup
import lxml.html
import random

In [ ]:
url = "https://suumo.jp/jj/bukken/ichiran/JJ012FC001/?ar=030&bs=011&cn=9999999&cnb=0&ekTjCd=&ekTjNm=&kb=1&kt=9999999&mb=0&mt=9999999&ta=13&tj=0&po=0&pj=1&pc=100&pn={}"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [ ]:
pg_list = [i for i in range(1, 226)]
property_list = []
regex = re.compile('.*property_unit.*')

In [ ]:
for page_num in pg_list:
    current_url = url.format(page_num)
    r = requests.get(current_url, headers=headers)
    
    
    soup = BeautifulSoup(r.content, "html.parser")
    properties = soup.find("div",{"id":"js-bukkenList"}).find_all("div", {"class":"property_unit"})
    more = soup.find("div",{"id":"js-bukkenList"}).find_all("div", {"class":"property_unit property_unit--osusume"})
    properties = set(properties + more)
    for prop in properties:
        #print(prop)
        name = prop.find_all("div",{"class":"dottable-line"})[0].find("dd").get_text()
        price = prop.find_all("div",{"class":"dottable-line"})[1].find("dd").get_text()
        address = prop.find_all("div",{"class":"dottable-line"})[2].find_all("dl")[0].find("dd").get_text()
        station = prop.find_all("div",{"class":"dottable-line"})[2].find_all("dl")[1].find("dd").get_text()
        size = prop.find_all("div",{"class":"dottable-line"})[3].find_all("dd")[0].get_text()
        plan = prop.find_all("div",{"class":"dottable-line"})[3].find_all("dd")[1].get_text()
        balcony = prop.find_all("div",{"class":"dottable-line"})[4].find_all("dd")[0].get_text()
        built_year = prop.find_all("div",{"class":"dottable-line"})[4].find_all("dd")[1].get_text()
        link = prop.find_all("h2")[0].find("a")["href"]
        property_list.append([name, price, address, station, size, plan, balcony, built_year, link])
    time.sleep(2)

In [ ]:
df = pd.DataFrame(property_list, columns = ['name', 'price', "address","station","size","plan","balcony","built_year","link"])

In [ ]:
df.to_csv("tokyo_apartment_property_basic_data.csv")

In [ ]:
base_url = "https://suumo.jp{}tokucho/?fmlg=t001"
url_list_detail_1 = [base_url.format(link) for link in list(df["link"])]
property_detail_list1 = []

In [ ]:
for i, prop_url in enumerate(url_list_detail_1):
    r = requests.get(prop_url, headers=headers)
    soup = BeautifulSoup(r.content, "html.parser")
    try:
        detail = soup.findAll('h3', text = re.compile('特徴ピックアップ'))[0].parent.findNext("div").get_text()
    except:
        detail = ""
    try:
        total_units = soup.find_all("table", {"summary":"表"})[0].findAll('div', text = re.compile('総戸数'))[0].parent.findNext("td").get_text()
    except:
        total_units = ""
    try:
        floor = soup.find_all("table", {"summary":"表"})[0].findAll('div', text = re.compile('所在階/構造・階建'))[0].parent.findNext("td").get_text()
    except:
        floor = ""
    property_detail_list1.append([prop_url, detail, total_units, floor])
    time.sleep(2)
    print(i + 1)
    if i > 150:
        if i % 150 == 0:
            time.sleep(60)

In [ ]:
df_detail = pd.DataFrame(property_detail_list1, columns = ["prop_url", "detail", "total_units", "floor"])

In [ ]:
df_detail.to_csv("tokyo_apartment_property_detail_data.csv")